# 第一阶段总结
经过第一阶段的整理，发现一下特点：
1. 连续两个涨停板的股票，大概率还会再涨
2. 大部分为小盘股票
3. 沉浸了半年以上忽然爆发
4. 会大涨大跌

# 第二阶段目标
了解结束的信号，以及结束后，这只股票的结果。
庄家什么时候收手，或者开启二次炒作。
1. 查找 20210301-20210501，20200301-20200501，20180601-20180801，被炒过的股票。
2. 查看他们的后期走势。

## 设想
1. 后期各个股票会归于沉寂
2. 抄完后，依然后很大波动，持续一段时间，最后归于沉寂
3. 长时间波动

In [18]:
import akshare as ak
import pandas as pd
import threading
from datetime import datetime
import os

In [19]:
print("-----开始-----")

-----开始-----


In [20]:
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, 'data')
file_list = os.listdir(data_dir)
print("文件数量：",len(file_list))

文件数量： 3150


In [21]:
# 读取csv 获得df
stock_df = pd.DataFrame()
for file in file_list:
    if file.endswith('.csv'):
        df = pd.read_csv(data_dir + '/' + file)
        stock_df = stock_df.append(df[['code','name','date','open','close','high','low','amount']])
stock_df = stock_df.reset_index(drop=True)
print("加载完毕！")
# 按照时间，
# 20180601-20180801 的数据 df1
# 20210301-20210501 的数据 df2
# 20200301-20200501 的数据 df3
# 计算涨幅，获得df1_1，df1_2
# 将df1_2向后移动一位，合并df1_1和df1_2
# 获取连续两天涨幅>0.1的股票

加载完毕！


In [22]:
# 添加索引列
stock_df.insert(0,'index_num',0)
stock_df['index_num'] = stock_df.groupby('code').index_num.transform(lambda x: range(len(x)))
# 计算涨幅
stock_df['rise_rate'] = (stock_df['close'] - stock_df['open']) / stock_df['open']
before_store_df = stock_df.copy(deep=True)
before_store_df.rename(columns={"rise_rate":"before_rise_rate"},inplace=True)
res_stock_df = pd.merge(stock_df,before_store_df[['code','index_num','before_rise_rate']],on=['code','index_num'],how='left')
print("计算增长率完毕！")

计算增长率完毕！


In [23]:
low_1 = datetime(2018, 6, 1)
high_1 = datetime(2018, 8, 1)
low_2 = datetime(2020, 3, 1)
high_2 = datetime(2020, 5, 1)
low_3 = datetime(2021, 3, 1)
high_3 = datetime(2021, 5, 1)

# 日期类型转换
res_stock_df['date']=pd.to_datetime(stock_df['date'],format='%Y-%m-%d')
res_stock_df.drop(columns=['index_num'],inplace=True)
stock_df_1 = res_stock_df[(res_stock_df['date']>low_1) & (res_stock_df['date']<high_1) & (res_stock_df['rise_rate']>0.1) & (res_stock_df['before_rise_rate']>0.1)]
stock_df_2 = res_stock_df[(res_stock_df['date']>low_2) & (res_stock_df['date']<high_2) & (res_stock_df['rise_rate']>0.1) & (res_stock_df['before_rise_rate']>0.1)]
stock_df_3 = res_stock_df[(res_stock_df['date']>low_3) & (res_stock_df['date']<high_3) & (res_stock_df['rise_rate']>0.1) & (res_stock_df['before_rise_rate']>0.1)]
print("数据筛选完毕！")

数据筛选完毕！


In [24]:
# 检查路径是否存在，如果不存在，就创建一个
dir = current_dir+'/continue_rise/'
if not os.path.exists(dir):
    os.makedirs(dir)
# 输出csv
if len(stock_df_1) > 0:
    stock_df_1.to_csv(dir+'/20180601_20180801_continue_rise.csv',index=False)
if len(stock_df_2) > 0:
    stock_df_2.to_csv(dir+'/20210301-20210501_continue_rise.csv',index=False)
if len(stock_df_3) > 0:
    stock_df_3.to_csv(dir+'/20200301-20200501_continue_rise.csv',index=False)
print("输出CSV完毕！")

输出CSV完毕！


In [25]:
print(len(stock_df_1))
print(len(stock_df_2))
print(len(stock_df_3))

509
941
922
